### Prompt:
You are an analyst for a fastfood restaurant. The company has tasked you to analyze their past transactions. Each row represents a single transaction/order from one of their ten (10) branches. The order is recorded as a __`string`__ which is formatted as: 

"`[quantity1][item_code1], [quantity2][item_code2], [quantity3][item_code3]`"

However, because of some technical error, the __number of combo meals purchased__ as well as the __total order value__ was lost, so we'll need to restore that.

### You are tasked to come up with the following:
1. Given the order column, count how many of each product was purchased
2. Given each product count, get the number of supposed combo meals purchased. <br><i>(You may update the product count when the combo meals are counted)</i>.
3. Calculate the total order value.
4. Determine which branch made the most revenue, and how much?

### The combo meals are detailed below:
1. Family combo: 4 burgers, 4 fries, 4 drinks
2. Big combo: 1 burgers, 1 fries, 1 drinks
3. Snack combo: 1 fries, 1 drinks

In [2]:
# 1. Given the order column, count how many of each product was purchased

import json
import pandas as pd
import numpy as np

def mayburger(x):
    if "Burger" in str(x):
        return x[int(x.index("Burger"))-3:int(x.index("Burger"))+6]

def mayfries(x):
    if "Fries" in str(x):
        return x[int(x.index("Fries"))-3:int(x.index("Fries"))+5]

def maydrink(x):
    if "Drink" in str(x):
        return x[int(x.index("Drink"))-3:int(x.index("Drink"))+5]

df=pd.DataFrame(pd.read_csv("food_transactions.csv"))
df.drop(columns=['Unnamed: 0'])

df['Burger']=df['order'].apply(mayburger)
df['Fries']=df['order'].apply(mayfries)
df['Drink']=df['order'].apply(maydrink)


df

,Unnamed: 0,date,branch,order,Burger,Fries,Drink
0,0,2023-06-26 21:36:39,Branch 3,"1S_Burger, 4S_Fries, 5S_Drink",1S_Burger,4S_Fries,5S_Drink
1,1,2023-12-22 10:57:56,Branch 9,"2S_Burger, 4S_Fries, 3S_Drink",2S_Burger,4S_Fries,3S_Drink
2,2,2023-12-14 14:07:51,Branch 8,"2S_Burger, 4S_Fries, 2S_Drink",2S_Burger,4S_Fries,2S_Drink
3,3,2023-11-01 02:10:22,Branch 4,"6S_Fries, 1S_Drink",None,6S_Fries,1S_Drink
4,4,2023-09-30 07:42:12,Branch 3,"5S_Burger, 6S_Drink",5S_Burger,None,6S_Drink
...,...,...,...,...,...,...,...
134770,134770,2023-12-11 04:21:04,Branch 8,"3S_Fries, 6S_Drink",None,3S_Fries,6S_Drink
134771,134771,2023-08-25 18:43:05,Branch 8,"7S_Burger, 6S_Fries",7S_Burger,6S_Fries,None
134772,134772,2023-09-24 14:15:58,Branch 10,"4S_Burger, 6S_Fries",4S_Burger,6S_Fries,None
134773,134773,2023-07-08 12:43:10,Branch 4,"7S_Burger, 6S_Fries, 5S_Drink",7S_Burger,6S_Fries,5S_Drink


In [3]:
# 2. Given each product count, get the number of supposed combo meals purchased.
# (You may update the product count when the combo meals are counted).
# The combo meals are detailed below:
# Family combo: 4 burgers, 4 fries, 4 drinks
# Big combo: 1 burgers, 1 fries, 1 drinks
# Snack combo: 1 fries, 1 drinks

def countcombosF(x):
    Family = 0
    if "Burger" in str(x):
        burgers = int(x[x.index("Burger")-3])
    else:
        burgers = 0
    if "Fries" in str(x):
        fries = int(x[x.index("Fries")-3])
    else:
        fries = 0
    if "Drink" in str(x):
        drink = int(x[x.index("Drink")-3])
    else:
        drink = 0
    while (burgers >= 4) & (fries >= 4) & (drink >= 4):
        burgers -= 4
        fries -= 4
        drink -= 4
        Family += 1
    if "Burger" in str(x):
        x = x.replace(x[x.index("Burger")-3], str(burgers))
    if "Fries" in str(x):
        x = x.replace(x[x.index("Fries")-3], str(fries))
    if "Drink" in str(x):
        x = x.replace(x[x.index("Drink")-3], str(drink))
    return Family

def updateorderF(x):
    Family = 0
    if "Burger" in str(x):
        burgers = int(x[x.index("Burger")-3])
    else:
        burgers = 0
    if "Fries" in str(x):
        fries = int(x[x.index("Fries")-3])
    else:
        fries = 0
    if "Drink" in str(x):
        drink = int(x[x.index("Drink")-3])
    else:
        drink = 0
    while (burgers >= 4) & (fries >= 4) & (drink >= 4):
        burgers -= 4
        fries -= 4
        drink -= 4
        Family += 1
    if "Burger" in str(x):
        x = x.replace(x[x.index("Burger")-3], str(burgers))
    if "Fries" in str(x):
        x = x.replace(x[x.index("Fries")-3], str(fries))
    if "Drink" in str(x):
        x = x.replace(x[x.index("Drink")-3], str(drink))
    return x

def countcombosB(x):
    Big = 0
    if "Burger" in str(x):
        burgers = int(x[x.index("Burger")-3])
    else:
        burgers = 0
    if "Fries" in str(x):
        fries = int(x[x.index("Fries")-3])
    else:
        fries = 0
    if "Drink" in str(x):
        drink = int(x[x.index("Drink")-3])
    else:
        drink = 0
    while (burgers >= 1) & (fries >= 1) & (drink >= 1):
        burgers -= 1
        fries -= 1
        drink -= 1
        Big += 1
    if "Burger" in str(x):
        x = x.replace(x[x.index("Burger")-3], str(burgers))
    if "Fries" in str(x):
        x = x.replace(x[x.index("Fries")-3], str(fries))
    if "Drink" in str(x):
        x = x.replace(x[x.index("Drink")-3], str(drink))
    return Big
    
def updateorderB(x):
    Big = 0
    if "Burger" in str(x):
        burgers = int(x[x.index("Burger")-3])
    else:
        burgers = 0
    if "Fries" in str(x):
        fries = int(x[x.index("Fries")-3])
    else:
        fries = 0
    if "Drink" in str(x):
        drink = int(x[x.index("Drink")-3])
    else:
        drink = 0
    while (burgers >= 1) & (fries >= 1) & (drink >= 1):
        burgers -= 1
        fries -= 1
        drink -= 1
        Big += 1
    if "Burger" in str(x):
        x = x.replace(x[x.index("Burger")-3], str(burgers))
    if "Fries" in str(x):
        x = x.replace(x[x.index("Fries")-3], str(fries))
    if "Drink" in str(x):
        x = x.replace(x[x.index("Drink")-3], str(drink))
    return x

def countcombosS(x):
    Snack = 0
    if "Burger" in str(x):
        burgers = int(x[x.index("Burger")-3])
    else:
        burgers = 0
    if "Fries" in str(x):
        fries = int(x[x.index("Fries")-3])
    else:
        fries = 0
    if "Drink" in str(x):
        drink = int(x[x.index("Drink")-3])
    else:
        drink = 0
    while (fries >= 1) & (drink >= 1):
        fries -= 1
        drink -= 1
        Snack += 1
    if "Burger" in str(x):
        x = x.replace(x[x.index("Burger")-3], str(burgers))
    if "Fries" in str(x):
        x = x.replace(x[x.index("Fries")-3], str(fries))
    if "Drink" in str(x):
        x = x.replace(x[x.index("Drink")-3], str(drink))
    return Snack

def updateorderS(x):
    Snack = 0
    if "Burger" in str(x):
        burgers = int(x[x.index("Burger")-3])
    else:
        burgers = 0
    if "Fries" in str(x):
        fries = int(x[x.index("Fries")-3])
    else:
        fries = 0
    if "Drink" in str(x):
        drink = int(x[x.index("Drink")-3])
    else:
        drink = 0
    while (fries >= 1) & (drink >= 1):
        fries -= 1
        drink -= 1
        Snack += 1
    if "Burger" in str(x):
        x = x.replace(x[x.index("Burger")-3], str(burgers))
    if "Fries" in str(x):
        x = x.replace(x[x.index("Fries")-3], str(fries))
    if "Drink" in str(x):
        x = x.replace(x[x.index("Drink")-3], str(drink))
    return x
    
df["Fam_combo"] = df["order"].apply(countcombosF)
df['order'] = df['order'].apply(updateorderF)
df['Big_combo'] = df['order'].apply(countcombosB)
df['order'] = df['order'].apply(updateorderB)
df['Snack_combo'] = df['order'].apply(countcombosS)
df['order'] = df['order'].apply(updateorderS)

df['Burger']=df['order'].apply(mayburger)
df['Fries']=df['order'].apply(mayfries)
df['Drink']=df['order'].apply(maydrink)

df

,Unnamed: 0,date,branch,order,Burger,Fries,Drink,Fam_combo,Big_combo,Snack_combo
0,0,2023-06-26 21:36:39,Branch 3,"0S_Burger, 0S_Fries, 1S_Drink",0S_Burger,0S_Fries,1S_Drink,0,1,3
1,1,2023-12-22 10:57:56,Branch 9,"0S_Burger, 0S_Fries, 0S_Drink",0S_Burger,0S_Fries,0S_Drink,0,2,1
2,2,2023-12-14 14:07:51,Branch 8,"0S_Burger, 2S_Fries, 0S_Drink",0S_Burger,2S_Fries,0S_Drink,0,2,0
3,3,2023-11-01 02:10:22,Branch 4,"5S_Fries, 0S_Drink",None,5S_Fries,0S_Drink,0,0,1
4,4,2023-09-30 07:42:12,Branch 3,"5S_Burger, 6S_Drink",5S_Burger,None,6S_Drink,0,0,0
...,...,...,...,...,...,...,...,...,...,...
134770,134770,2023-12-11 04:21:04,Branch 8,"0S_Fries, 3S_Drink",None,0S_Fries,3S_Drink,0,0,3
134771,134771,2023-08-25 18:43:05,Branch 8,"7S_Burger, 6S_Fries",7S_Burger,6S_Fries,None,0,0,0
134772,134772,2023-09-24 14:15:58,Branch 10,"4S_Burger, 6S_Fries",4S_Burger,6S_Fries,None,0,0,0
134773,134773,2023-07-08 12:43:10,Branch 4,"0S_Burger, 0S_Fries, 0S_Drink",0S_Burger,0S_Fries,0S_Drink,1,1,0


In [4]:
# 3. Calculate the total order value.
jsondf = pd.DataFrame(pd.read_json("price_list.json"))
jsondf

# def getfirst(x):
#     return x[x.index("Burger")-3]
    
df["Big_combo_price"] = df["Big_combo"] * jsondf.iloc[0,0]
df["Fam_combo_price"] = df["Fam_combo"] * jsondf.iloc[1,0]
# df["Burger_combo_price"] = df["Burger"].apply(getfirst) * jsondf.iloc[2,0]
df


,Unnamed: 0,date,branch,order,Burger,Fries,Drink,Fam_combo,Big_combo,Snack_combo,Big_combo_price,Fam_combo_price
0,0,2023-06-26 21:36:39,Branch 3,"0S_Burger, 0S_Fries, 1S_Drink",0S_Burger,0S_Fries,1S_Drink,0,1,3,169,0
1,1,2023-12-22 10:57:56,Branch 9,"0S_Burger, 0S_Fries, 0S_Drink",0S_Burger,0S_Fries,0S_Drink,0,2,1,338,0
2,2,2023-12-14 14:07:51,Branch 8,"0S_Burger, 2S_Fries, 0S_Drink",0S_Burger,2S_Fries,0S_Drink,0,2,0,338,0
3,3,2023-11-01 02:10:22,Branch 4,"5S_Fries, 0S_Drink",None,5S_Fries,0S_Drink,0,0,1,0,0
4,4,2023-09-30 07:42:12,Branch 3,"5S_Burger, 6S_Drink",5S_Burger,None,6S_Drink,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
134770,134770,2023-12-11 04:21:04,Branch 8,"0S_Fries, 3S_Drink",None,0S_Fries,3S_Drink,0,0,3,0,0
134771,134771,2023-08-25 18:43:05,Branch 8,"7S_Burger, 6S_Fries",7S_Burger,6S_Fries,None,0,0,0,0,0
134772,134772,2023-09-24 14:15:58,Branch 10,"4S_Burger, 6S_Fries",4S_Burger,6S_Fries,None,0,0,0,0,0
134773,134773,2023-07-08 12:43:10,Branch 4,"0S_Burger, 0S_Fries, 0S_Drink",0S_Burger,0S_Fries,0S_Drink,1,1,0,169,599


In [5]:
# 4. Determine which branch made the most revenue, and how much?

df.groupby('branch')['Branch Revenue'].sum
df.sort_values('Branch Revenue', ascending=False)


KeyError: 'Column not found: Branch Revenue'